In [1]:
import numpy as np
import tensorflow as tf
import cv2 as cv
import math
import threading
from scipy.optimize import least_squares

C:\Users\pooni\AppData\Local\Programs\Python\Python310\lib\site-packages\requests\__init__.py:78: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({0}) or chardet ({1}) doesn't match a supported "


AttributeError: module 'collections' has no attribute 'MutableMapping'

In [ ]:
LAYERS = [
    ('Conv1',1),
    ('block_1_project',1),
    ('block_2_project',1),
    ('block_3_project',1),
    ('block_4_project',1),
    ('block_5_project',1),
    ('block_6_project',1),
    ('block_7_project',1),
    ('block_8_project',1),
    ('block_9_project',1),
    ('block_10_project',1),
    ('block_11_project',1),
    ('block_12_project',1),
    ('block_13_project',1),
    ('block_14_project',1),
    ('block_15_project',1),
    ('block_16_project',1),
    ('Conv_1', 1)
]

def content_error(content_image_output, generated_image_output):
    m, nh, nw, nc = content_image_output.get_shape().as_list()

    ac = tf.reshape(content_image_output, shape=[m, -1, nc])
    ag = tf.reshape(generated_image_output, shape=[m, -1, nc])

    j_content = tf.divide(tf.reduce_sum(tf.square(tf.subtract(ac, ag))), (4.0 * nh * nw * nc))

    return j_content

def construct_model(size, layers=LAYERS):
    temp = tf.keras.applications.MobileNetV2(include_top=False,
                                       input_shape=size + (3,),
                                       weights='imagenet')
    temp.trainable = False
    output = [temp.get_layer(layer[0]).output for layer in layers]
    temp_model = tf.keras.Model([temp.input], output)
    return temp_model

def find_params(model, image):
    image = cv.resize(image, (224, 224))
    image = np.array([image])

    params = model(image)

    return params

def find_feature_loss(params_one, params_two):
    feature_loss = []
    for i,j in zip(params_one,params_two):
        feature_loss.append(content_error(i, j))
    feature_loss = np.array(feature_loss)
    res = np.prod(feature_loss)
    return res


In [14]:
def find_quality(image):
    my_kernel = np.array([[-1,-1,-1],[-1,8,-1],[-1,-1,-1]])
    image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    image = cv.filter2D(image, -1, cv.flip(my_kernel, -1), borderType=cv.BORDER_CONSTANT)
    res = np.std(image)
    return res

In [15]:
def make_mask(first_image, second_image, flag):
    height = second_image.shape[0]
    width = second_image.shape[1]
    window = 200
    offset = first_image.shape[1] - window
    mask = np.zeros((height, width))

    if flag == 'l':
        mask[:, offset-window:offset+window] = np.tile( np.linspace(1, 0, 2*window).T, (height, 1))
        mask[:, :offset-window] = 1
    elif flag == 'r':
        mask[:, offset-window:offset+window] = np.tile( np.linspace(0, 1, 2*window).T, (height, 1))
        mask[:, offset+window:] = 1
    return cv.merge([mask, mask, mask])

def overlap(first_image, result_image):
    height = first_image.shape[0]
    width = first_image.shape[1]
    left_mask = make_mask(first_image, result_image, 'l')
    right_mask = make_mask(first_image, result_image, 'r')

    result_zero = result_image[:, :] != [0,0,0]
    first_zero = first_image[:, :] != [0,0,0]

    left_mask = left_mask * result_zero
    right_mask[0:height, 0:width] = right_mask[0:height, 0:width] * first_zero

    left_mask = left_mask + ~result_zero
    right_mask[0:height, 0:width] = right_mask[0:height, 0:width] + ~first_zero

    result_image = result_image * right_mask
    first_image = first_image * left_mask[0:height, 0:width]

    result_image[0:height, 0:width] = result_image[0:height, 0:width] + first_image


    return result_image

In [19]:
def resize(image, resolution):
    height = image.shape[0]
    width = image.shape[1]

    if height > resolution:
        n = height / resolution
        image = cv.resize(image, (int(width / n), resolution))

    return image

def loss(h, base_pts, res_pts):
    h = h.reshape(3,3)
    res1 = cv.perspectiveTransform(res_pts.reshape(-1,1,2), h).reshape(-1,2)

    h_inv = np.linalg.inv(h)
    res2 = cv.perspectiveTransform(base_pts.reshape(-1, 1, 2), h_inv).reshape(-1, 2)

    return ((base_pts - res1) + (res_pts - res2)).flatten()

# def loss(h, base_pts, res_pts):
#     h = h.reshape(3,3)
#     res1 = cv.perspectiveTransform(res_pts.reshape(-1,1,2), h).reshape(-1,2)
#
#     h_inv = np.linalg.inv(h)
#     res2 = cv.perspectiveTransform(base_pts.reshape(-1, 1, 2), h_inv).reshape(-1, 2)
#
#     return ((np.power((np.power((base_pts[:, 0], res1[:, 0]), 2) + np.power((base_pts[:, 1], res1[:, 1]), 2)), 0.5)) + (np.power((np.power((res_pts[:, 0], res2[:, 0]), 2) + np.power((res_pts[:, 1], res2[:, 1]), 2)), 0.5))).flatten()

class JoinOne:
    def find_keypoint(self, image):
        image_gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
        sift = cv.SIFT_create()
        (key_point, feature) = sift.detectAndCompute(image_gray, None)
        key_point = np.float32([k.pt for k in key_point])
        return key_point, feature

    def find_homography(self, train_key_point, query_key_point, train_feature, query_feature, ratio, threshold):
        matcher = cv.BFMatcher()
        match_points = matcher.knnMatch(train_feature, query_feature, k=2)
        valid_match_points = []

        for m in match_points:
            if m[0].distance < m[1].distance * ratio:
                valid_match_points.append((m[0].trainIdx, m[0].queryIdx))
        if len(valid_match_points) > 4:
            train_valid_points = np.float32([train_key_point[i] for (_, i) in valid_match_points])
            query_valid_points = np.float32([query_key_point[i] for (i, _) in valid_match_points])

            (H, status) = cv.findHomography(train_valid_points, query_valid_points, cv.RANSAC, threshold)
            res_h = least_squares(loss, H.flatten(), args=(query_valid_points, train_valid_points), method='lm')
            H = res_h.x.reshape(3,3)

        return valid_match_points, H, status

    def stitch_images(self, train_image, query_image, ratio=0.7, threshold=4.0):
        resolution = 3840

        train_image = resize(train_image, resolution)
        query_image = resize(query_image, resolution)

        image_one = train_image
        image_two = query_image

        # image_two_width = image_two.shape[1]
        # image_two_height - image_two.shape[0]
        #
        # divide = int((3*image_two_width)/5)

        # image_two_first_section = image_two[:, 0:divide]
        # image_two_second_section = image_two[:, divide:image_two_width]



        (train_key_point, train_feature) = self.find_keypoint(train_image)
        (query_key_point, query_feature) = self.find_keypoint(query_image)

        (valid_match_points, H, status) = self.find_homography(train_key_point, query_key_point, train_feature, query_feature, ratio, threshold)

        width = image_one.shape[1] + image_two.shape[1]
        height = image_two.shape[0]

        result_image = cv.warpPerspective( image_one, H, (width, height))
        result_image = overlap(image_two, result_image)

        rows, cols = np.where(result_image[:, :, 0] != 0)
        min_row, max_row = min(rows), max(rows) + 1
        min_col, max_col = min(cols), max(cols) + 1
        final_result = result_image[min_row:max_row, min_col:max_col, :]

        return final_result

In [17]:
# instantaneous variables
params = None
frame = None
quality = None
panorama = []
current_best = 0
count = 0

model = construct_model((224, 224))


# function to join image
def joinImages():
    global panorama, current_best, count
    print("<<<<<----- Joining images ----->>>>>")
    panorama = JoinOne().stitch_images(current_best[0], panorama)
    panorama = cv.normalize(panorama, None, 0, 255, cv.NORM_MINMAX).astype('uint8')
    cv.imwrite(f'intermediate_results/b {count}.jpg', panorama)
    print("<<<<<----- Joining complete ----->>>>>")

# function to find params
def run_find_params():
    global params, frame, model, count
    print(f"<< finding params for: {count} >>")
    params = find_params(model, frame)
    return params

# function to find quality
def run_find_quality():
    global  quality, frame, count
    print(f"<< finding quality for: {count} >>")
    quality = find_quality(frame)
    return quality


def make_panorama(video_name):
    global params, frame, quality, panorama, current_best, count
    # static values
    threshold = 2

    # similarity model

    # main variables

    base_params = 0


    video = cv.VideoCapture(video_name)

    # thread to join images
    join_image = None



    while video.isOpened():

        # instantaneous variables
        params = None
        quality = None

        count += 1
        ret, frame = video.read()

        if not ret:
            join_image.join()
            join_image = threading.Thread(target=joinImages)
            join_image.start()
            break

        frame = cv.rotate(frame, cv.ROTATE_180)

        print(f"<< finding stats for : {count} >>")

        # thread to find params
        find_p = threading.Thread(target=run_find_params)

        # thread to find quality
        find_q = threading.Thread(target=run_find_quality)

        find_p.start()
        find_q.start()

        find_p.join()
        find_q.join()


        if base_params != 0:
            feature_loss = find_feature_loss(params, base_params)

            if feature_loss * math.pow(10,23)  < threshold:

                if quality > current_best[1]:
                    current_best = [frame, quality]
            else:
                if len(panorama) != 0:
                    if join_image is not None: join_image.join()
                    join_image = threading.Thread(target=joinImages)
                    join_image.start()

                else:
                    panorama = current_best[0]
                base_params = params
                current_best = [frame, quality]


        else:
            base_params = params
            current_best = [frame, quality]

        print(f"Completed frame: {count}")
        print("-------------------------")

    video.release()
    join_image.join()
    return panorama




In [18]:
res = make_panorama("E:/Ansh Poonia/Desktop/computer vision/video/12.mp4")

<< finding stats for : 1 >>
<< finding params for: 1 >>
<< finding quality for: 1 >>
Completed frame: 1
-------------------------
<< finding stats for : 2 >>
<< finding params for: 2 >>
<< finding quality for: 2 >>
Completed frame: 2
-------------------------
<< finding stats for : 3 >>
<< finding params for: 3 >>
<< finding quality for: 3 >>
Completed frame: 3
-------------------------
<< finding stats for : 4 >>
<< finding params for: 4 >>
<< finding quality for: 4 >>
Completed frame: 4
-------------------------
<< finding stats for : 5 >>
<< finding params for: 5 >>
<< finding quality for: 5 >>
Completed frame: 5
-------------------------
<< finding stats for : 6 >>
<< finding params for: 6 >>
<< finding quality for: 6 >>
Completed frame: 6
-------------------------
<< finding stats for : 7 >>
<< finding params for: 7 >>
<< finding quality for: 7 >>
Completed frame: 7
-------------------------
<< finding stats for : 8 >>
<< finding params for: 8 >>
<< finding quality for: 8 >>
Compl


KeyboardInterrupt



In [ ]:
cv.imwrite("final_results/seven.jpg", res)